# Habituation to weakly non-Gaussian odor backgrounds
Look at a case with three odors. Compare BioPCA and IBCM models for habituation and new odor detection. 

## Some notes on the setting
### Saturation function on IBCM neurons
Since the fluctuations are still fast compared to the IBCM neurons, we do not really need saturation functions to prevent numerical divergences (while still using a large time step). Also, the model converges fast enough that we don't need the Law and Cooper, 1994 modification. 

### Background process
We simulate a zero-mean Ornstein-Uhlenbeck process $g_\gamma$ with mean $\langle c \rangle$ and variance $\sigma^2$ at steady-state, then we set the actual odor concentrations to be $c_\gamma = 10^{g_\gamma}$. This ensures that the odor concentrations are log-normal and have a third moment. For each $\gamma$, the moments are

$$ \langle c \rangle = 10^{\langle g \rangle + \frac12 \sigma^2 \ln{10}} $$
$$ \mathrm{Var}[c] = \left(10^{\sigma^2 \ln{10}} - 1 \right) 10^{2 \langle g \rangle + \sigma^2 \ln{10}} $$
$$ \langle (c - \langle{c}\rangle)^3 \rangle = \mathrm{Var}[c]^{3/2} \left(10^{\sigma^2 \ln{10}} + 2\right) \sqrt{10^{\sigma^2 \ln{10}} - 1} \,\, . $$

For multiple odors, the set of $\{g_{\gamma}\}$ is a multivariate O-U process as defined in Gardiner's Handbook. But in practice, we only consider independent odors, each can thus be thought of as a scalar O-U process. 


### General case of the Ornstein-Uhlenbeck process
The multivariate Langevin equation for the Ornstein-Uhlenbeck process is:

$$ d\vec{x} = -A \vec{x}(t) dt + B dW(t) $$

where $\frac{dW}{dt} = \vec{\eta}(t)$, a vector of gaussian white noise (independent components), $A$ and $B$ are matrices. Assume the matrix $A$ is normal and can be diagonalized as $A = U D u^\dagger$, $D = \mathrm{diag}(\lambda^1, ..., \lambda^n)$. For a deterministic initial condition $\vec{x}(t_0) = \vec{x}_0$, the general solution is that $\vec{x}(t)$ follows a multivariate normal distribution, with mean and variance given by

$$ \langle \vec{x}(t) \rangle = U\mathrm{e}^{-D(t-t_0)}U^{\dagger} \vec{x}_0 $$
$$ \langle \vec{x}(t) \vec{x}(t)^T \rangle = U J(t, t_0) U^\dagger $$

where the components of $J$ are 

$$ J^{ij}(t, t_0) = \left(\frac{U^\dagger B B^T U}{\lambda^i + \lambda^j} \right)^{ij} \left(1 - e^{-(\lambda^i + \lambda^j)(t - t_0)}  \right) $$


The stationary distribution of $\vec{x}$ is

$$ \vec{x}^* \sim \mathcal{N} \left(\vec{0}, U^{ik} \left(\frac{B B^T}{\lambda^k + \lambda^l} \right)^{kl} (U^\dagger)^{lj} \right) \,\, .$$

### Exact numerical simulation, general case
To simulate a realization of this process exactly, we use a trick suggested by Gillespie in the univariate case (which only works for the Ornstein-Uhlenbeck process because it's linear and gaussian). We iteratively take $\vec{x}(t)$ as the initial condition of the evolution up to $\vec{x}(t + \Delta t)$, the distribution of which is

$$ \vec{x}(t + \Delta t) \sim \mathcal{N}\left( U e^{-D \Delta t}U^\dagger \vec{x}(t) , U J(t + \Delta t, t) U^\dagger \right) $$

which can be rewritten using the following property of multivariate normal distributions: if $\vec{n} \sim \mathcal{N}(\vec{0}, \mathbb{1})$, then $\vec{x} = \vec{\mu} + \Psi \vec{n} \sim \mathcal{N}(\vec{\mu}, \Psi \Psi^T)$ ($\Psi$ is the Cholesky decomposition of the desired covariance matrix). This property is easily demonstrated by computing $\langle \vec{x} \rangle$ and $\langle \vec{x} \vec{x}^T \rangle$ and using the linearity of multivariate normal distributions. For our update rule, this gives

$$ \vec{x}(t + \Delta t) = U e^{-D \Delta t}U^\dagger \vec{x}(t) + \mathrm{Chol}\left[U J(t + \Delta t, t) U^\dagger \right] \cdot \vec{n} $$

where $\vec{n}$ is a vector of standard normal(0, 1) samples. The matrices $U e^{-D \Delta t}U^\dagger$ and $\mathrm{Chol}\left[U J(t + \Delta t, t) U^\dagger \right]$ can be computed only once and applied repeatedly to the $\vec{x}(t)$ obtained in sequence and the $\vec{n}$ drawn at each iteration. The Cholesky decomposition of $UJU^\dagger$ is not obviously expressed in terms of $B$, because the possibly different $\lambda^i$ values mix up components. 


### Simple case and exact simulation of it
If $A$ is diagonal, the $U$ matrices are just identity matrices and disappear, but the Cholesky decomposition of $J(t + \Delta t, t)$ is still not obvious. More explicit expressions can be obtained in the simplifying case where $A$ is proportional to the identity matrix, i.e., all components of $\vec{x}$ have the same fluctuation time scale. 

Let's say that $A =  \frac{1}{\tau} \mathbb{1}$, where $\tau$ is the fluctuation time scale ($\lambda^i = \tau \,\, \forall i$). Then, the matrix $J$ simplifies to 

$$J(t, t_0) = \frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right)  BB^T  $$

and its Cholesky decomposition is simply $\sqrt{\frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right) } B$. Hence, the distribution of $\vec{x}(t)$ at any time since $t_0$ (deterministic initial condition $\vec{x}_0$) is

$$ \vec{x}(t) \sim \mathcal{N} \left(e^{-(t-t_0)/\tau} \vec{x}_0, \frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right)  BB^T  \right) $$

The stationary distribution is simply the above with the exponential factors set to 0. The update rule from $\vec{x}(t)$ to $\vec{x}(t + \Delta t)$ to simulate a realization of the process is nicer as well:

$$ \vec{x}(t + \Delta t) = e^{-\Delta t / \tau} \vec{x}(t) + \sqrt{\frac{\tau}{2} \left(1 - e^{-2\Delta t/\tau}  \right)} B \cdot \vec{n} $$

where $\vec{n} \sim \mathcal{N}(\vec{0}, \mathbb{1})$ is a vector of independent standard normal samples.

As before, we can compute once the (scalar) factor $e^{-\Delta t / \tau}$ and the . This is exact for any $\Delta t$, there is no increase in accuracy by decreasing $\Delta t$. You just choose the $\Delta t$ resolution at which you want to sample the realization of the process. 

### Symmetric choices for correlations
We want all pairs of $\nu_\alpha$ to have the same correlation. More specifically, we want to force a Pearson correlation coefficient of $0 < \rho < 1$ between any pair of $\nu$s. We suppose all background components have the same individual variance $\sigma^2$. The corresponding covariance matrix we want for the steady-state distribution is

$$ \Sigma = \sigma^2 \begin{pmatrix}
    1 & \rho & \ldots & \rho \\
    \rho & 1 & \ldots & \rho \\
    \ldots & \ldots & \ldots & \ldots \\
    \rho & \rho & \ldots & 1
\end{pmatrix} $$

If we apply Cholesky decomposition to get $\Sigma = \Psi \Psi^T$, then $\sqrt{\tau/2} B = \Psi$, since the steady-state covariance of the Ornstein-Uhlenbeck process is, in this simplified case, $\frac{\tau}{2} BB^T$. The $M_B$ coefficient in the update rule is then

$$ M_B = \sqrt{\tau/2(1 - e^{-2 \Delta t/\tau})}B = \sqrt{(1 - e^{-2 \Delta t/\tau})} \Psi $$

The other coefficient is just

$$ M_A = e^{-\Delta t / \tau} \mathbb{1} $$

## Imports

In [1]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, sys
if ".." not in sys.path:
    sys.path.insert(1, "..")

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues,
    ibcm_saddle_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor, 
    compute_optimal_matrices
)
from modelfcts.checktools import (
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_logou_kinputs,
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard

ModuleNotFoundError: No module named 'modelfcts'

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)
plt.rcParams["axes.facecolor"] = (1,1,1,0)  # transparent background

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "optimal": "Optimal", 
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:light green",
    "optimal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 25  # Half the real number for faster simulations
n_components = 6  # Number of background odors

inhib_rates = [0.0001, 0.00002]  # alpha, beta

# Simulation duration
duration = 320000.0
deltat = 1.0
skp = 20

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
update_fct = update_logou_kinputs

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x6fe5a179ffe6f22c0d705e844176ab8e)
back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Initial background vector and initial nu values
# Log-normal concentrations, nus are the logs of concentrations
ln10 = np.log(10.0)
averages_nu = -0.5*np.ones(n_components)  # Average of log(c); for c < 1, these averages are < 0
init_nu = averages_nu.copy()
init_bkvec = np.exp(averages_nu*ln10).dot(back_components)
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## Compute the matrices in the Ornstein-Uhlenbeck update equation
# Update matrix for the mean term: 
# Exponential decay with time scale tau_nu over time deltat
tau_nu = 2.0  # Fluctuation time scale of the background nu_gammas (same for all)
update_mat_A = np.identity(n_components)*np.exp(-deltat/tau_nu)

# Steady-state covariance matrix
sigma2 = 0.09
correl_rho = 0.0
steady_covmat = correl_rho * sigma2 * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat[np.eye(n_components, dtype=bool)] = sigma2  # diagonal: ones


# Mean and variance of the concentrations themselves
# Using moments of log-normal: https://en.wikipedia.org/wiki/Log-normal_distribution
mean_nu_lnbase = averages_nu.mean() * ln10
vari_nu_lnbase = sigma2 * ln10**2
lognorm_mean = np.exp(mean_nu_lnbase + vari_nu_lnbase/2.0)
lognorm_vari = (np.exp(vari_nu_lnbase) - 1.0)*np.exp(2*mean_nu_lnbase + vari_nu_lnbase)
# Third centered moment: rom skewness, multiply by its variance**3
lognorm_skewness = (np.exp(vari_nu_lnbase) + 2)*np.sqrt(np.exp(vari_nu_lnbase) - 1)
lognorm_thirdmom = lognorm_skewness * lognorm_vari**1.5
moments_conc_lognorm = [lognorm_mean, lognorm_vari, lognorm_thirdmom]

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat = np.linalg.cholesky(steady_covmat)
update_mat_B = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat

back_params = [update_mat_A, update_mat_B, back_components, averages_nu]
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.00075  # 0.000001 = 1e-6
tau_avg_ibcm = 200
coupling_eta_ibcm = 0.5/n_i_ibcm
decay_relative_ibcm = 0.0  # dummy
k_c2bar_avg = 1.0  # dummy
ssat_ibcm = 50.0  # dummy
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "intrator", 
    "decay": False
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.1*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions])

In [ ]:
# Run the IBCM simulations
sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back_list, 
                ibcm_rates, inhib_rates, back_params, duration, 
                deltat, seed=simul_seed, noisetype="normal",  
                skp=skp, **ibcm_options
)

(tser_ibcm, 
 nuser_ibcm, 
 bkvecser_ibcm, 
 mser_ibcm, 
 cbarser_ibcm, 
 thetaser_ibcm,
 wser_ibcm, 
 yser_ibcm) = sim_results

### IBCM habituation analysis

In [ ]:
# Calculate cgammas_bar and mbars
transient = 160000 // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, back_components)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. 
mean_g = np.mean(averages_nu)
mean_conc = 10.0**(mean_g + 0.5*sigma2*ln10)
variance_conc = (10.0**(sigma2*ln10) - 1.0) * 10.0**(2*mean_g + sigma2*ln10)
thirdmoment_conc = variance_conc**1.5 * (10.0**(sigma2*ln10) + 2.0) * np.sqrt(10.0**(sigma2*ln10) - 1.0)

# Compare to previously computed analytical moments (moments_conc_lognorm) and to numerical values
# The nuser are the zero-meaned gaussian variables: need to add back average nu first. 
fullconcser = np.exp((nuser_ibcm + averages_nu) * ln10)
mean_conc_sim = np.mean(fullconcser)  # all odors i.i.d., can average over them. 
variance_conc_sim = np.mean((fullconcser - mean_conc_sim)**2)
thirdmoment_sim = np.mean((fullconcser - mean_conc_sim)**3)
moments_conc = [mean_conc, variance_conc, thirdmoment_conc]
moments_conc_sim = [mean_conc_sim, variance_conc_sim, thirdmoment_sim]
# These three lists of moments should all agree. First two exactly, last should be close
print(moments_conc_lognorm)
print(moments_conc)
print(moments_conc_sim)

# Analytical prediction
hs_hn = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1, lambd=lambd_ibcm)
hsum_fixed, h2sum_fixed = hs_hn[2:]
hs_hn = hs_hn[:2]
h_specif, h_nonspecif = hs_hn

# Also check the saddle point where all h_gammas are equal, I suspect the model goes there first. 
saddle_h = fixedpoint_thirdmoment_exact(moments_conc, n_components, 0, lambd=lambd_ibcm)[0]
print(saddle_h)

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 1.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, back_components, hs_hn, specif_gammas)

In [ ]:
# Plot the cbar2_avg term throughout
cbar2_avg_ser = moving_average(cbarser_ibcm*cbarser_ibcm, kernelsize=tau_avg_ibcm)
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
fig, ax = plt.subplots()
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm[:-tau_avg_ibcm], cbar2_avg_ser[:-tau_avg_ibcm, i], 
            color=neurons_cmap[i])
ax.set(xlabel="Time (x1000)", ylabel=r"$\bar{c}^2$ moving average")
ax.axhline(h2sum_fixed * variance_conc, ls="--", color="k", lw=1.5)
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=160000 // skp)
ax.legend(loc="center right")
# Compare to exact analytical fixed point solution
ax.axhline(h_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(h_nonspecif, ls="-", color="k", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
ax.axhline(saddle_h, ls=":", color="grey", 
           label="Saddle point?")
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = fullconcser - mean_conc
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
i_colors = sns.color_palette(n_colors=n_dimensions)
for j in range(n_i_ibcm):
    for i in range(n_dimensions):
        axes.flat[j].axhline(analytical_w[i, j], color=i_colors[i], ls="--", 
                             lw=0.5, alpha=0.7, zorder=-j*n_dimensions-i)
    #axes.flat[j].set_ylim([axes.flat[j].get_ylim()[0], np.amax(analytical_w[:, j])*1.1])
plt.show()
plt.close()

### Stability of average fixed points
Check the eigenvalues of the jacobian for one neuron, for every possible specificity. There are $2^{n_B}$ possibilities: choosing specific or not for each odor

In [ ]:
all_max_eigenvalues = ibcm_all_largest_eigenvalues(
    moments_conc, ibcm_rates, back_components, m3=1.0, cut=1e-16
)

In [ ]:
fig, ax = plt.subplots()
ibcm_specif_keys = list(all_max_eigenvalues.keys())
ibcm_eig_values = np.asarray([all_max_eigenvalues[a] for a in ibcm_specif_keys])
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k")
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", mfc="b", mec="b", 
        ls="none", label="One odor", ms=8)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=6)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity")
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
# Also curious about the eigenvalues at the saddle point
all_saddle_eigenvalues = ibcm_saddle_eigenvalues(
    moments_conc, ibcm_rates, back_components, m3=1.0, cut=1e-16
)

# Note that there's always an eigenvalue for the threshold Theta. So the true number of eigenvalues is 1 + N_B
# Now if N_B are positive, only one is negative, I'm not sure this is even a saddle

In [ ]:
fig, ax = plt.subplots()
reals, imags = np.real(all_saddle_eigenvalues), np.imag(all_saddle_eigenvalues)
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k")
scaleup = 1e3
sizes = np.arange(6, len(all_saddle_eigenvalues)+6)
for i in range(len(reals)):
    ax.plot(reals[i]*scaleup, imags[i]*scaleup, marker="o", 
            mfc=(0, 0, 0, 1.0-0.1*i), mec="k", ls="none", ms=sizes[i])
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{saddle}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{saddle}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
from modelfcts.ibcm_analytics import lambda_pca_equivalent

In [ ]:
# BioPCA model parameters
n_i_pca = n_components  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 0.0005  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.8   # With range of 0.8, breaks degeneracy, find actual PCA. But here, want to show it doesn't always converge? 
lambda_max_pca = lambda_pca_equivalent(hs_hn, moments_conc, n_components, inhib_rates, verbose=True)
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
init_synapses_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="normal", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 yser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
plt.imshow(lser_pca[-1])
plt.colorbar()
plt.title("L matrix")
plt.show()
plt.close()

In [ ]:
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, yser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_pca, bkvecser_pca, yser_pca, skp=10)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
plt.show()
plt.close()

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="normal", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, yser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
yser_ideal = bkvecser_ibcm * ideal_factor

### Optimal W manifold learning

In [ ]:
# This matrix depends on new odor concentrations, so define them here
# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
new_test_concs *= moments_conc[0]
n_new_concs = len(new_test_concs)

# Compute optimal W matrix for all new odors possible
dummy_rgen = np.random.default_rng(0x6e3e2886c30163741daaaf7c8b8a00e6)
new_odors_from_distrib = generate_odorant([int(1e5), n_dimensions], dummy_rgen, lambda_in=0.1)
new_odors_from_distrib /= l2_norm(new_odors_from_distrib)[:, None]
optimal_matrices = compute_optimal_matrices(back_components, new_odors_from_distrib, moments_conc, new_test_concs)

# Use the W matrix for the lowest concentration to inhibit the background
yser_optimal = bkvecser_ibcm - bkvecser_ibcm.dot(optimal_matrices[0].T)

## Model comparison for background inhibition

In [ ]:
ynorm_series = {
    "ibcm": l2_norm(yser_ibcm), 
    "biopca": l2_norm(yser_pca), 
    "avgsub": l2_norm(yser_avg), 
    "none": l2_norm(bkvecser_ibcm), 
    "ideal": l2_norm(yser_ideal),
    "optimal": l2_norm(yser_optimal)
}
std_options = dict(kernelsize=2001, boundary="free")
mean_options = dict(kernelsize=2001, boundary="free")
std_series = {
    a: np.sqrt(moving_var(ynorm_series[a], **std_options)) for a in ynorm_series
} 
mean_series = {
    a: moving_average(ynorm_series[a], **mean_options) for a in ynorm_series
}

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes = axes.flatten()
for model in std_series.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    axes[0].plot(tser_ibcm / 1000, mean_series[model], **props)
    axes[1].plot(tser_ibcm / 1000, std_series[model], **props)
ynorm_string = r"$\|\vec{s}\|$"
axes[0].set_ylabel(r"PN activity norm, " + ynorm_string)
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Standard deviation " + ynorm_string)
axes[0].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
fig.set_size_inches(4.5, 2.5*2)
plt.show()
plt.close()

## Model comparison for background tagging after habituation
We create a projection matrix, then compute the tag assigned to the background after inhibition by each habituation model, over time. Hopefully, only IBCM inhibits enough to see tags go to zero. 

In [ ]:
# Common parameters
n_kc = 1000
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)

In [ ]:
# Computing background tag lengths at various time points for each model
yser_dict = {
    "ibcm": yser_ibcm, 
    "biopca": yser_pca, 
    "avgsub": yser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": yser_ideal, 
    "optimal": yser_optimal, 
    "orthogonal": np.zeros(yser_ibcm.shape)
}
tag_length_series = {a: np.zeros(tser_ibcm.shape[0]) for a in yser_dict.keys()}
for a in yser_dict.keys():
    for i in range(0, tag_length_series[a].shape[0]):
        if bkvecser_ibcm[i].max() > 0:
            tag = project_neural_tag(yser_dict[a][i], bkvecser_ibcm[i], 
                                 proj_mat, **projection_arguments)
        else:
            tag = (1,)*int(projection_arguments["kc_sparsity"]*n_kc)
        tag_length_series[a][i] = len(tag)
tag_length_series_smooth = {a: moving_average(tag_length_series[a], **mean_options)
                            for a in tag_length_series}

In [ ]:
fig, ax = plt.subplots()
for model in yser_dict.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    ax.plot(tser_ibcm / 1000, tag_length_series_smooth[model], **props)
ynorm_string = r"$\|\vec{s}\|$"
ax.set_ylabel(r"Tag length, $\mathrm{card}(z)$")
ax.set_xlabel("Time (x1000 steps)")
ax.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
plt.show()
plt.close()

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 100
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

# New odor concentrations, defined above

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
# sample_ss_distrib_thirdmoment(means_nu, covmat_nu, epsil, size=1, rgen=None)
# First sample Gaussian
conc_samples = rgen_meta.multivariate_normal(averages_nu, steady_covmat, 
                        size=n_test_times*(n_back_samples-1))  # Shaped [sample, component]
conc_samples = 10.0**conc_samples
back_samples = conc_samples.dot(back_components)
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# Containers for s vectors of each model
mixture_yvecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in yser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in yser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in yser_dict.keys()}
jaccard_backs = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in yser_dict.keys()}
back_tags = [project_neural_tag(b, b,proj_mat, **projection_arguments) for b in back_components]

In [ ]:
# Compute ideal reduction factor for each concentration
ideal_factors = [compute_ideal_factor(c, moments_conc[:2], [n_components, n_dimensions], 
                    generate_odorant, (dummy_rgen,)) for c in new_test_concs]
for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_yvecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_yvecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_yvecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_yvecs["none"][i, j, k] = mixtures
            mixture_yvecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, mixtures, new_test_concs[k], 
                ideal_factors[k], **avg_options
            )
            mixture_yvecs["optimal"][i, j, k] = mixtures - mixtures.dot(optimal_matrices[k].T)
            mixture_yvecs["orthogonal"][i, j, k] = x_new_ort
            for l in range(n_back_samples):
                for mod in mixture_yvecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_yvecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_yvecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_yvecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)
                    jaccard_backs[mod][i, j, k, l] = max((jaccard(mix_tag, b) for b in back_tags))

In [ ]:
ideal_factors

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm", "optimal"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("../figures/detection/compare_models_onerun_lognormal_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Similarity with background odors vs with new odor: should be as low as possible
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm", "optimal"]
for m in models:  # Plot IBCM last
    # Take median across test times
    all_jacs = np.median(jaccard_scores[m], axis=1)
    all_back_jacs = np.median(jaccard_backs[m], axis=1)
    for i in range(n_new_concs):
        axes[i].plot(all_back_jacs[:, i].flatten(), all_jacs[:, i].flatten(),
            marker="o", ls="none", ms=2.0, alpha=0.4, 
            label=model_nice_names.get(m, m), color=model_colors.get(m))
# Draw a diagonal on each graph

for i in range(n_new_concs):
    ylim = [0, 0.95]
    xlim = [0, 0.95]
    axes[i].plot(xlim, ylim, ls="--", color="grey", lw=1.0)
    axes[i].set_xlim(xlim)
    axes[i].set_ylim(ylim)

# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard with background\n(lower is better)")
    axes[i].set_ylabel("Jaccard with new odor\n(higher is better)")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("../figures/detection/compare_models_onerun_lognormal_jaccards_back_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Distance to new odor
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm", "optimal"]
all_medians = []
for m in models:  # Plot IBCM last
    all_distances = (mixture_yvecs[m] 
         - new_test_concs[None, None, :, None, None]*new_odors[:, None, None, None, :])
    all_norms = l2_norm(all_distances.reshape(-1, n_dimensions))
    all_medians.append(np.median(all_norms))
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_norms,
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            all_medians[-1], ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    axes[i].set_xlim([0.0, 2.0*max(all_medians)])
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel(r"Distance to new odor, $\|\vec{s} - \vec{x}_{\mathrm{new}}\|$")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("../figures/detection/compare_models_onerun_non-gaussian_ynorm_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

# Save some results for further plotting
In particular, save excerpt of concentration time series. 

In [ ]:
# We also want an unskipped nu time series for the figure, run a dummy simulation for its background
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration/2.0, deltat,
                seed=simul_seed, noisetype="normal", skp=1, **avg_options
)
_, nuser_noskp, _, _, _ = sim_results

In [ ]:
import json
results_filename = os.path.join("..", "results", "for_plots", "sample_lognormal_simulation.npz")
save_skp = 10
# Include some results from BioPCA too. 
np.savez_compressed(
    results_filename, 
    tser=tser_ibcm[::save_skp], 
    nuser=nuser_noskp,   # Can get conc_ser and bkvecser from that and components
    mbarser=mbarser[::save_skp],  # Can get cbars_gamma_ser from that and components
    cbarser=cbarser_ibcm[::save_skp],
    wser=wser_ibcm[::save_skp], 
    yser=yser_ibcm[::save_skp], 
    true_pca_vals=true_pca[0],
    true_pca_vecs=true_pca[1], 
    learnt_pca_vals=learnt_pca[0][::save_skp],
    learnt_pca_vecs=learnt_pca[1][::save_skp],
    off_diag_l_pca=off_diag_l_avg_abs[::save_skp],
    align_error_pca=align_error_ser[::save_skp],
    yser_pca=yser_pca[::save_skp],
    yser_optimal=yser_optimal[::save_skp], 
    back_vecs=back_components, 
    hs_hn=hs_hn,
    saddle_h=saddle_h,
    analytical_w=analytical_w,  # Can get analytical yser from that, hs_hn, and specif_gammas
    specif_gammas=specif_gammas, 
    skp=skp*save_skp,
    averages_nu0=averages_nu, 
    sigma2=sigma2, 
    ibcm_eig_values=ibcm_eig_values, 
    new_test_concs=new_test_concs,
    moments_conc=moments_conc
)

# Save Jaccard similarities for this run
jaccards_filename = os.path.join("..", "results", "for_plots", "jaccards_onerun_lognormal_simulation.npz")
all_jacs2 = {m:jaccard_scores[m] for m in jaccard_scores.keys()}
all_jacs2.pop("ideal")
all_back_jacs2 = {m+"_back": jaccard_backs[m] for m in jaccard_backs.keys()}
all_back_jacs2.pop("ideal_back")
all_jacs2.update(all_back_jacs2)
np.savez_compressed(
    jaccards_filename, 
    **all_jacs2
)

with open(os.path.join("..", "results", "for_plots", "ibcm_eigenvalues_keys_lognormal_example.json"), "w") as f:
    json.dump(ibcm_specif_keys, f)

In [ ]:
all_jacs2.keys()